<a href="https://colab.research.google.com/github/YuMan-Tam/deep-hedging-demo/blob/master/deep_hedging_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title <font color='Blue'>**Overheads**</font>

# Author: Yu-Man Tam
# Last updated: 3/3/2020

# Reference: Deep Hedging (2019, Quantitative Finance) by Buehler et al.
# https://www.tandfonline.com/doi/abs/10.1080/14697688.2019.1571683

import sys, os
sys.path.insert(0, os.getcwd() + "/lib")

# Linear algebra, finance, and machine learning libraries
import numpy as np
import QuantLib as ql
import tensorflow as tf

from itertools import repeat
import pickle
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model
import tensorflow.keras.backend as K

# For plots and presentation
from tqdm import trange, tqdm
from tqdm.keras import TqdmCallback
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# For Jupyter notebooks
from IPython.display import clear_output
from IPython.core.display import display, HTML

from stochastic_processes import BlackScholesProcess
from instruments import European_Call
from utilities import train_test_split
from deep_hedging import Deep_Hedging_Model
from loss_metrics import Entropy, CVaR

# Tensorflow settings
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

%load_ext autoreload


clear_output()

print("\nFinish loading all necessary libraries!\n")


Finish loading all necessary libraries!



**Import all neccessary python, quantitative finance, and machine learning software libraries.**

In [2]:
#@title <font color='Blue'>**User Inputs**</font>

# Geometric Brownian Motion.
N = 30 # Number of time steps (in days)

S0 = 100.0 # Stock price at time = 0
sigma = 0.2 # Implied volatility
risk_free = 0.0 # Risk-free rate
dividend = 0.0 # Continuous dividend yield

Ktrain = 1*(10**5) # Size of training sample.
Ktest_ratio = 0.2 # Fraction of training sample as testing sample.

# European call option (short).
strike = S0
payoff_func = lambda x: -np.maximum(x - strike, 0.0)
calculation_date = ql.Date.todaysDate()
maturity_date = ql.Date.todaysDate() + N

# Day convention.
day_count = ql.Actual365Fixed() # Actual/Actual (ISDA)

# Proportional transaction cost.
epsilon = np.power(2.0,-8)*0.0                 

# Information set (in string)
# Choose from: S, log_S, normalized_log_S (by S0)
information_set = "normalized_log_S"

# Loss function
# loss_type = "CVaR" (Expected Shortfall) -> loss_param = alpha 
# loss_type = "Entropy" -> loss_param = lambda
loss_type = "Entropy"
loss_param = 1.0

# Neural network (NN) structure
m = 15 # Number of neurons in each hidden layer.
d = 3 # Number of hidden layers (Note including input nor output layer)         

# Other NN parameters
lr = 1e-2 # Learning rate
batch_size=256 # Batch size
epochs=50 # Number of epochs

use_batch_norm = False
kernel_initializer = "he_uniform"

activation_dense = "leaky_relu"
activation_output = "sigmoid"
final_period_cost = False

# Other control flags for development purpose.
mc_simulator = "QuantLib" # "QuantLib" or "Numpy"

flag_load_simulation_data = True
flag_load_model_simple = True
flag_load_model_recurrent = False
flag_load_BS_delta = True
flag_load_BS_asym = True
flag_show_fig = False
flag_plot_S_range = "uniform" # "uniform" or "percentile"
flag_load_PnL_DH_Simple_list = True
flag_presenation_mode = True

# Output directories
simulation_data_dir = "./data/"
subdir_hyper_param = "/".join((str(epsilon),activation_dense,activation_output,str(m),str(d),str(batch_size),""))
model_data_dir ="./data/" + subdir_hyper_param
fig_dir = "./figure/"+ subdir_hyper_param

print("Finish loading all user inputs!\n")

Finish loading all user inputs!



**Provide input parameters for Monte Carlo simulation, call option, transaction cost, loss function, and deep hedging algorithm.**

In [3]:
#@title <font color='Blue'>**Monte Carlo Simulation - Generate Random Paths of Stock Prices**</font>

# Create a directory if the output directories do not exist.
for dir in (simulation_data_dir, model_data_dir, fig_dir):
	try:
		os.makedirs(dir)
	except:
		pass
		
# Length of one time-step (as fraction of a year).
nobs = int(Ktrain*(1+Ktest_ratio)) # Training + Testing

dt = day_count.yearFraction(calculation_date,calculation_date + 1) 
maturity = N*dt # Maturities (in the unit of a year)

stochastic_process = BlackScholesProcess(s0 = S0, sigma = sigma, \
                      risk_free = risk_free, dividend = dividend, day_count=day_count)

simulation_data_file = simulation_data_dir +"stock_" + str(int(np.log10(Ktrain))) + ".npy"
if not flag_load_simulation_data:
	if mc_simulator is "QuantLib":
		S = stochastic_process.gen_path(maturity, N, nobs)
	elif mc_simulator is "Numpy":
		# Only for geometric Brownian motion
		randn = (risk_free - dividend - sigma ** 2 / 2.0) * dt + \
								sigma * np.random.normal(0, np.sqrt(dt), size=(nobs, N))
		S = np.hstack((np.ones((nobs,1))*np.log(S0),randn))
		S = np.exp(np.cumsum(S,axis=1))
		
	np.save(simulation_data_file, S)
else:	
	try:
	  print("Check if there is usable data from previous simulation...\n")
	  S = np.load(simulation_data_file)
	  print("Successfully loaded data!!!\n")
	except:
		print("There is no simulation data!!!")

Check if there is usable data from previous simulation...

Successfully loaded data!!!



In [4]:
#@title <font color='Blue'>**Prepare data to be fed into the deep hedging algorithm.**</font>
payoff_T = payoff_func(S[:,-1]) # Payoff of the call option

trade_set =  np.stack((S),axis=1) # Trading set

if information_set is "S":
  I =  np.stack((S),axis=1) # Information set
elif information_set is "log_S":
  I =  np.stack((np.log(S)),axis=1)
elif information_set is "normalized_log_S":
	I =  np.stack((np.log(S/S0)),axis=1)
	
# Structure of xtrain:
#   1) Trade set: [S]
#   2) Information set: [S] 
#   3) payoff (dim = 1)
x_all = []
for i in range(N+1):
	x_all += [trade_set[i,:,None]]
	if i != N:
		x_all += [I[i,:,None]]
x_all += [payoff_T[:,None]]

# Split the entire sample into a training sample and a testing sample.
test_size = int(Ktrain*Ktest_ratio)
[xtrain, xtest] = train_test_split(x_all, test_size=test_size)
[S_train, S_test] = train_test_split([S], test_size=test_size)
[option_payoff_train, option_payoff_test] = \
    train_test_split([x_all[-1]], test_size=test_size)

# Convert the training sample into tf.Data format (same as xtrain).
training_dataset = tf.data.Dataset.from_tensor_slices(tuple(xtrain))
training_dataset = training_dataset.cache()

# Obtain Black-Scholes price, delta, and PnL
call = European_Call()
price_BS = call.get_BS_price(S = S_test[0], sigma = sigma, risk_free = risk_free, \
									dividend = dividend, K = strike, exercise_date = maturity_date, \
									calculation_date = calculation_date, day_count = day_count, dt = dt)
delta_BS = call.get_BS_delta(S = S_test[0], sigma = sigma, risk_free = risk_free, \
									dividend = dividend, K = strike, exercise_date = maturity_date, \
									calculation_date = calculation_date, day_count = day_count, dt = dt)

print("Finish preparing training and testing data!")

Finish preparing training and testing data!


In [0]:
#@title <font color='Blue'>**Run the Deep Hedging Algorithm (Simple Network)!**</font>
%autoreload 2

draw_freq = 15
strategy_type = "simple"
epsilon = 0.008
optimizer = Adam(learning_rate=lr)

# PnL Heding using Black-Scholes delta
PnL_BS =  call.get_BS_PnL(S=S_test[0], payoff=payoff_func(S_test[0][:,-1]), \
                  delta=delta_BS, dt=dt, risk_free = risk_free, \
									final_period_cost=final_period_cost, epsilon=epsilon)

# Setup and compile the model
model_simple = Deep_Hedging_Model(N=N, d=d+2, m=m, risk_free=risk_free, \
                      dt = dt, strategy_type=strategy_type, epsilon = epsilon, \
											use_batch_norm = use_batch_norm, kernel_initializer = kernel_initializer, \
											activation_dense = activation_dense, activation_output = activation_output, \
											final_period_cost = final_period_cost)
# Accelerate the code using tf.function. 
model_simple_func = tf.function(model_simple)

# Plot live graph!!

# For interactive graph
matplotlib.use('TkAGG') 
plt.ion()
plt.close("all")

fig_PnL = plt.figure(dpi= 150, facecolor='w')
fig_PnL.suptitle("Black-Scholes PnL vs Deep Hedging PnL \n", \
			fontweight="bold")
ax = fig_PnL.add_subplot()
ax.set_title("Simple Network Structure with epsilon = " + str(epsilon), \
			fontsize=8)
ax.set_xlabel("PnL")
ax.set_ylabel("Frequency")
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

fig_PnL.show()

# Certainty equivalent as defined in Buehler's paper. 
certainty_equiv = tf.Variable(0.0)

plot_data = training_dataset.shuffle(Ktrain).batch(batch_size)
for num_epoch in range(epochs):
	num_batch = 0
	for mini_batch in training_dataset.shuffle(Ktrain).batch(batch_size):
		num_batch = num_batch + 1
		
		with tf.GradientTape() as tape:
			wealth = model_simple_func(mini_batch)
			loss = Entropy(wealth, certainty_equiv, loss_param)
		
		if num_batch % draw_freq == 0:
			PnL_DH_Simple = model_simple_func(xtest).numpy().squeeze()
					
			ax.cla()
			textstr = '\n'.join((
				r'$Epoch=%.1d$' % (num_epoch+1, ),
				r'$Batch=%.1d$' % (num_batch, ),
				r'$Loss=%.3f$' % (loss, )))
			ax.text(0.05, 0.75, textstr, transform=ax.transAxes, fontsize=11,
							verticalalignment='top', bbox=props)
			
			ax.hist((PnL_BS+price_BS[0,0],PnL_DH_Simple + price_BS[0,0]), \
						range = (PnL_BS.min()+price_BS[0,0], PnL_BS.max()+price_BS[0,0]), \
						bins=30, label=["Black-Scholes PnL", "Deep Hedging PnL"])
			ax.legend(loc="upper left")
			
			fig_PnL.canvas.draw()
		grads = tape.gradient(loss, model_simple.trainable_weights)
		optimizer.apply_gradients(zip(grads, model_simple.trainable_weights))

plt.ioff()